In [1]:
import cleaning
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import plotly.express as px
import plotly.graph_objs as go
import utils

from ipywidgets import widgets
from plotly import tools
from plotly.offline import iplot
from plotly import colors

In [2]:
def load_recipes(index_range):
    with pd.HDFStore("all_recipes.h5", "r") as store:
        where_clause = f"(index >= {index_range[0]}) & (index <= {index_range[1]})"
        core = store.select("/core", where=where_clause)
        ing = store.select("/ingredients", where=where_clause)
    df = core.join(ing)
    return df

In [3]:
brewersfriend_indices = [330790, 403157]
brewtoad_indices = [258423, 330789]

In [4]:
bf = load_recipes(brewersfriend_indices)
bt = load_recipes(brewtoad_indices)

In [5]:
df = pd.concat([bf, bt], axis=0).sort_index()

In [6]:
def generate_metrics(df):
    df["ferm_scaled"] = utils.scale_ferm(df)
    df["hop_scaled"] = utils.scale_hop(df)
    df["efficiency"] = cleaning.clean_efficiency(df["efficiency"])
    df["ferm_yield"] = cleaning.clean_ferm_yield(df)
    df["ibu"] = utils.ibu(df, utilization_factor=3.75)
    df["abv"] = utils.abv(df)
    df["srm"] = utils.srm(df)

In [7]:
generate_metrics(df)

In [8]:
metrics = df.groupby(df.index).first()

In [101]:
def generate_hist(df, style):
    tmp = df.loc[df["style_name"] == style]
    ibu = tmp["ibu"].clip(0, 200)
    abv = tmp["abv"].clip(0, 15)
    hist, x_edge, y_edge = np.histogram2d(ibu.values, abv.values, bins=20, range=[[0,200],[0,20]], density=True)
    x_mid = (x_edge[1:] + x_edge[:-1])/2.
    y_mid = (y_edge[1:] + y_edge[:-1])/2.
    return hist, x_mid, y_mid

# Creating a Dynamic Plot

* Define all the pieces of what you will show
  * The graph itself (e.g. scatter plot)
  * Any buttons, sliders, etc.
  * Data sources
* Define any validator and/or response functions
  * Validators make sure inputs are OK
  * Response functions make sure the graph changes appropriately - they are fed to `observe()` methods for each widget (could have different or the same response functions for each widget)
* Show your plot
  * Layout all your items, including the plot itself

In [102]:
# We want a contour plot, and we would like to select the group of beer that is being contoured based in user input

contour_traces = []

g = go.FigureWidget(
    data=contour_traces,
    layout=go.Layout(
        title=dict(text='IBU vs ABV')
    )
)

dropdown = widgets.SelectMultiple(
    description="Style Name",
    options=sorted(metrics.style_name.unique().tolist()),
)

In [103]:
def validator_template():
    return True


def response(change):
    if validator_template():
        new_traces = []
        for style_name in dropdown.value:
            hist, x_mid, y_mid = generate_hist(metrics, style_name)
            new_traces.append(
                go.Contour(
                    name=style_name,
                    x=x_mid,
                    y=y_mid,
                    z=hist,
                    contours_coloring='lines',
                    line_width=2,
                    contours={"start":0.005, "end": 0.02},
                ),
            )
        layout = {
            "title": dict(text='ABV vs IBU'),
            "xaxis": dict(title="IBU"),
            "yaxis": dict(title="ABV"),
        }

        g.update(dict1={"data": new_traces, "layout": layout}, overwrite=True)

dropdown.observe(response)

In [104]:
widgets.VBox([dropdown, g])

In [13]:
import datetime
import numpy as np
import pandas as pd

import plotly.graph_objects as go


In [14]:
df = pd.read_csv(
    'https://raw.githubusercontent.com/yankev/testing/master/datasets/nycflights.csv')
df = df.drop(df.columns[[0]], axis=1)

In [15]:
month = widgets.IntSlider(
    value=1.0,
    min=1.0,
    max=12.0,
    step=1.0,
    description='Month:',
    continuous_update=False
)

use_date = widgets.Checkbox(
    description='Date: ',
    value=True,
)

textbox = widgets.Dropdown(
    description='Airline:   ',
    value='DL',
    options=df['carrier'].unique().tolist()
)

origin = widgets.Dropdown(
    options=list(df['origin'].unique()),
    value='LGA',
    description='Origin Airport:',
)

# Assign an empty figure widget with two traces
trace1 = go.Histogram(x=df['arr_delay'], opacity=0.75, name='Arrival Delays')
trace2 = go.Histogram(x=df['dep_delay'], opacity=0.75, name='Departure Delays')
g = go.FigureWidget(data=[trace1, trace2],
                    layout=go.Layout(
                        title=dict(
                            text='NYC FlightDatabase'
                        ),
                        barmode='overlay'
                    ))

In [16]:
def validate():
    if origin.value in df['origin'].unique() and textbox.value in df['carrier'].unique():
        return True
    else:
        return False


def response(change):
    if validate():
        if use_date.value:
            filter_list = [i and j and k for i, j, k in
                           zip(df['month'] == month.value, df['carrier'] == textbox.value,
                               df['origin'] == origin.value)]
            temp_df = df[filter_list]

        else:
            filter_list = [i and j for i, j in
                           zip(df['carrier'] == 'DL', df['origin'] == origin.value)]
            temp_df = df[filter_list]
        x1 = temp_df['arr_delay']
        x2 = temp_df['dep_delay']
        with g.batch_update():
            g.data[0].x = x1
            g.data[1].x = x2
            g.layout.barmode = 'overlay'
            g.layout.xaxis.title = 'Delay in Minutes'
            g.layout.yaxis.title = 'Number of Delays'


origin.observe(response, names="value")
textbox.observe(response, names="value")
month.observe(response, names="value")
use_date.observe(response, names="value")

In [17]:
container = widgets.HBox(children=[use_date, month])
container2 = widgets.HBox([origin, textbox])
widgets.VBox([container,
              container2,
              g])